In [1]:
import pandas as pd

In [4]:
data_path = "./mimic-iii-clinical-database-1.4"

In [5]:
# Load the necessary tables
admissions = pd.read_csv(f"{data_path}/ADMISSIONS.csv")
noteevents = pd.read_csv(f"{data_path}/NOTEEVENTS.csv")
diagnoses_icd = pd.read_csv(f"{data_path}/DIAGNOSES_ICD.csv")
d_icd_diagnoses = pd.read_csv(f"{data_path}/D_ICD_DIAGNOSES.csv")

C:\Users\NourH\AppData\Local\Temp\ipykernel_5780\2913900359.py:3: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  noteevents = pd.read_csv(f"{data_path}/NOTEEVENTS.csv")


In [10]:
d_icd_diagnoses = d_icd_diagnoses.drop(columns=["ROW_ID"])

In [6]:
# Step 1: Filter only discharge summaries
discharge_notes = noteevents[noteevents["CATEGORY"] == "Discharge summary"]

In [7]:
# Step 2: Join discharge notes with admissions to get structured info
notes_with_adm = pd.merge(discharge_notes, admissions, on=["SUBJECT_ID", "HADM_ID"], how="inner")

In [8]:
# Step 3: Join with diagnosis codes
notes_with_diag = pd.merge(notes_with_adm, diagnoses_icd, on=["SUBJECT_ID", "HADM_ID"], how="left")

In [11]:
# Step 4: Join with ICD code descriptions
notes_with_diag = pd.merge(notes_with_diag, d_icd_diagnoses, on="ICD9_CODE", how="left")

In [15]:
# Step 5: Select a few columns to display
selected_columns = notes_with_diag[[
    "SUBJECT_ID", "CATEGORY", "HADM_ID", "ADMITTIME", "DISCHTIME", "TEXT", "ICD9_CODE", "SHORT_TITLE", "LONG_TITLE"
]]

In [20]:
# Show a few example records
pd.set_option('display.max_colwidth', 50)
selected_columns.head(3)

,SUBJECT_ID,CATEGORY,HADM_ID,ADMITTIME,DISCHTIME,TEXT,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,22532,Discharge summary,167853.0,2151-07-16 14:29:00,2151-08-04 19:10:00,Admission Date: [**2151-7-16**] Dischar...,01193,Pulmon TB NOS-micro dx,"Pulmonary tuberculosis, unspecified, tubercle ..."
1,22532,Discharge summary,167853.0,2151-07-16 14:29:00,2151-08-04 19:10:00,Admission Date: [**2151-7-16**] Dischar...,4254,Prim cardiomyopathy NEC,Other primary cardiomyopathies
2,22532,Discharge summary,167853.0,2151-07-16 14:29:00,2151-08-04 19:10:00,Admission Date: [**2151-7-16**] Dischar...,42731,Atrial fibrillation,Atrial fibrillation


In [21]:
# Step 4: Keep only the first ICD code per admission
notes_with_diag = notes_with_diag.sort_values(["SUBJECT_ID", "HADM_ID", "SEQ_NUM"]) 
notes_with_diag = notes_with_diag.drop_duplicates(subset=["HADM_ID"], keep="first")

In [23]:
# Step 5: Merge with ICD-9 code descriptions
notes_with_diag = pd.merge(notes_with_diag, d_icd_diagnoses, on="ICD9_CODE", how="left")

In [28]:
# Step 6: Select relevant columns
final_df = notes_with_diag[[
    "SUBJECT_ID", "HADM_ID", "ICD9_CODE", "TEXT"
]]

In [31]:
# Show example records
pd.set_option('display.max_colwidth', 500)
final_df.head(3)

,SUBJECT_ID,HADM_ID,ICD9_CODE,TEXT
0,3,145834.0,0389,Admission Date: [**2101-10-20**] Discharge Date: [**2101-10-31**]\n\nDate of Birth: [**2025-4-11**] Sex: M\n\nService: Medicine\n\nCHIEF COMPLAINT: Admitted from rehabilitation for\nhypotension (systolic blood pressure to the 70s) and\ndecreased urine output.\n\nHISTORY OF PRESENT ILLNESS: The patient is a 76-year-old\nmale who had been hospitalized at the [**Hospital1 190**] from [**10-11**] through [**10-19**] of [**2101**]\nafter undergoing a left femoral-AT bypass graft a...
1,4,185777.0,042,"Admission Date: [**2191-3-16**] Discharge Date: [**2191-3-23**]\n\nDate of Birth: [**2143-5-12**] Sex: F\n\nService:\n\nCHIEF COMPLAINT: Shortness of breath and fevers.\n\nHISTORY OF PRESENT ILLNESS: The patient is a 47-year-old\nfemale with a history of human immunodeficiency virus (last\nCD4 count 42 and a viral load of 65,000), cirrhosis,\ndiabetes, and hypothyroidism presented with eight days of\nfevers to 104, chills, shortness of breath, cough, dyspnea on\nexertion, and ..."
2,6,107064.0,40391,"Admission Date: [**2175-5-30**] Discharge Date: [**2175-6-15**]\n\nDate of Birth: Sex: F\n\nService:\n\n\nADMISSION DIAGNOSIS: End stage renal disease, admitted for\ntransplant surgery.\n\nHISTORY OF PRESENT ILLNESS: The patient is a 65 year-old\nwoman with end stage renal disease, secondary to malignant\nhypertension. She was started on dialysis in [**2174-2-7**]. She currently was on peritoneal dialysis and appears\nto be doing well. She has a history of gas..."


In [ ]:
print(f"Number of records: {final_df.shape[0]}")
print(f"Number of features: {final_df.shape[1]}")

Number of records: 52726


In [36]:
# Keep only 'TEXT' and 'ICD9_CODE' columns
final_df = notes_with_diag[["TEXT", "ICD9_CODE"]]

In [37]:
final_df = final_df.dropna(subset=["TEXT", "ICD9_CODE"])

In [40]:
# Display a few records
pd.set_option('display.max_colwidth', 500)
final_df.head(3)

,TEXT,ICD9_CODE
0,Admission Date: [**2101-10-20**] Discharge Date: [**2101-10-31**]\n\nDate of Birth: [**2025-4-11**] Sex: M\n\nService: Medicine\n\nCHIEF COMPLAINT: Admitted from rehabilitation for\nhypotension (systolic blood pressure to the 70s) and\ndecreased urine output.\n\nHISTORY OF PRESENT ILLNESS: The patient is a 76-year-old\nmale who had been hospitalized at the [**Hospital1 190**] from [**10-11**] through [**10-19**] of [**2101**]\nafter undergoing a left femoral-AT bypass graft a...,0389
1,"Admission Date: [**2191-3-16**] Discharge Date: [**2191-3-23**]\n\nDate of Birth: [**2143-5-12**] Sex: F\n\nService:\n\nCHIEF COMPLAINT: Shortness of breath and fevers.\n\nHISTORY OF PRESENT ILLNESS: The patient is a 47-year-old\nfemale with a history of human immunodeficiency virus (last\nCD4 count 42 and a viral load of 65,000), cirrhosis,\ndiabetes, and hypothyroidism presented with eight days of\nfevers to 104, chills, shortness of breath, cough, dyspnea on\nexertion, and ...",042
2,"Admission Date: [**2175-5-30**] Discharge Date: [**2175-6-15**]\n\nDate of Birth: Sex: F\n\nService:\n\n\nADMISSION DIAGNOSIS: End stage renal disease, admitted for\ntransplant surgery.\n\nHISTORY OF PRESENT ILLNESS: The patient is a 65 year-old\nwoman with end stage renal disease, secondary to malignant\nhypertension. She was started on dialysis in [**2174-2-7**]. She currently was on peritoneal dialysis and appears\nto be doing well. She has a history of gas...",40391


In [42]:
print(f"Number of records: {final_df.shape[0]}")
print(f"Number of features: {final_df.shape[1]}")

Number of records: 52722
Number of features: 2


In [48]:
final_df.to_csv("./data/discharge_icd_clean.csv", index=False)